In [7]:
import secrets
import string
import pandas as pd
import hashlib
import base64
import os

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes



In [8]:
csv_path = "../modified_data/hotel_booking_token.csv"
csv_output_path = "../modified_data/hotel_booking_token_rsaemail.csv"
public_path = "public_key.pem"

In [9]:

def generate_keys():
    if os.path.exists(public_path):
      return
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    # Generate public key
    public_key = private_key.public_key()
    
    # Save the private key
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))
    
    # Save the public key
    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))
# Usage
generate_keys()


In [10]:

# Load public key
with open("public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(
        f.read(),
        backend=default_backend()
    )
# Load private key
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None,
        backend=default_backend()
    )

def encrypt_message(message):
    # Encrypt message
    encrypted = base64.b64encode(public_key.encrypt(
        message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )).decode('utf-8')
    return encrypted

def decrypt_message(encrypted_message):
    # Decrypt message
    original_message = private_key.decrypt(
        base64.b64decode(encrypted_message),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return original_message.decode()

encrypted_message = encrypt_message("Hello, this is a secret message!")
print("Encrypted:", encrypted_message)
decrypted_message = decrypt_message(encrypted_message)
print("Decrypted:", decrypted_message)

Encrypted: Rw+xRL7Z33LzLPWZrLTC3ex9VUma77ANxibx9Hw4/SyG0oe/6RYhEVYb+QDX4Dww0Y9fMT1VA/fntruZDWP0Y/KHvcxVxcdaaS9nhn3h59/M/wWWMRINqz7q8fLbJ3CKl+r92u3LzzpGN6zs4wWyt8heW63hesJoGWrX1YuChQK9tsIsgHWbnP3GP52CqmSKk2mwn+o9cwWnfMMmYbhE2FDiVB2w9bgX/2TPCUvjRh/soQDdwThnEvHmzkJ3DEvimacZ9vxJY5FPOY6k+S9YKGp7LB3L3VzqF4QLH+yI7t9p1mu26aY489YHYVQ9icPneO5G6sP5b+FwwHwgWn9meQ==
Decrypted: Hello, this is a secret message!


In [11]:
total = 0
file_path = csv_path
simple_data = pd.read_csv(file_path)

total=len(simple_data)


In [12]:
encrpyted_emails = []

simple_data['email'] = simple_data['email'].apply(lambda x: encrypt_message(x))
print(simple_data['email'].values[0])
print(decrypt_message(simple_data['email'].values[0]))

simple_data.to_csv(csv_output_path, index=False)

ndtGCR85zrq+HWJNjIJtteUtJ8Vc5AM8ylLmMmG9WKKpNoH/3KepIc+EkIj714+QP5rNW2F8AS03pM5Y4Q9fLNpLCJF+xuhYefansultQOC1c4GkQXCO6UQDZ3yw25w395qQF6Uy2GF5lhyzDGn0gqgT4plsc/HhvDRhAb5BUMZBaQ+1cEgFEqiMR8PxB0hRrKEa/iMY84KfSj+beM3aHbWk08SIShJX5DSdWelvTMhEh6AgjW0uR4VpYGn770Hir8gfqViaVb5qlnA5tpkOXJFgj/B6aAY+bhhLJ2AN09azTKmpBH4XvSLWFN+bQP8bhFazvatBFD0yqsU8RtdRKg==
Ernest.Barnes31@outlook.com
